In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import requests

In [2]:
def download_one_file_of_raw_data_from_nyc_gov_website(year:int, month:int) -> Path:
    """"""
    url_string = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(url_string)

    if response.status_code==200:
        print(f'Writing rides_{year}-{month:02d}.parquet', end='\r')
        path = f'../data/raw/rides_{year}-{month:02d}.parquet'
        open(path, 'wb').write(response.content)
        return path
    
    else:
        raise Exception(f'{url_string} is not available')

In [3]:
# fetch and write parquet data files; one per year-month
years = [2022]
months = list(np.arange(1,13))

for y in years:
    for m in months:
        download_one_file_of_raw_data_from_nyc_gov_website(y,m)
print('                                    ')
print(f'done!')

                                    
done!


In [4]:
# concatenate parquet tables into a pandas-df
# this could be done faster; check https://jpweytjens.be/read-multiple-files-with-pandas-fast/
raw_data_dir = Path('../data/raw')
full_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in raw_data_dir.glob('*.parquet')
)

In [5]:
# check full df
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39656098 entries, 0 to 3183766
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            flo

In [8]:
# slice relevant data and check
rides_df = full_df.copy()[['tpep_pickup_datetime', 'PULocationID']]

rides_df.rename(columns={
    'tpep_pickup_datetime': 'pickup_datetime',
    'PULocationID': 'pickup_location_id',
}, inplace=True)

rides_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39656098 entries, 0 to 3183766
Data columns (total 2 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_datetime     datetime64[ns]
 1   pickup_location_id  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 907.7 MB


In [9]:
rides_df['pickup_datetime'].describe()

count                         39656098
mean     2022-07-05 04:04:16.211664640
min                2001-01-01 00:03:14
25%                2022-04-07 21:59:21
50%         2022-07-01 06:36:25.500000
75%                2022-10-04 20:17:12
max                2023-04-18 14:30:05
Name: pickup_datetime, dtype: object